# Test 03 — Line Follow (straight + curves)

This uses `fast_hi_wonder`:
- `InfraredSensors().read_sensor_data()`
- `Motors().set_velocity(speed, direction_deg, yaw_rate)`

Controller:
- Compute lateral error from sensor pattern
- Apply P + D control to yaw
- Recover when line is lost by turning in the last known direction

⚠️ Put the robot on the floor with a clear line track before running.

In [ ]:
from pathlib import Path
import sys

def add_repo_root():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            if str(p) not in sys.path:
                sys.path.insert(0, str(p))
            print('Repo root:', p)
            return p
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_root()

In [ ]:
import time
from fast_hi_wonder import Motors, InfraredSensors

motors = Motors()
ir = InfraredSensors()

def stop():
    # safest stop we have
    try:
        motors.reset_motors()
    except Exception:
        pass

print('OK: Motors + InfraredSensors loaded')

## 1) Sensor sanity check

Run this first. You should see a list of 0/1 (or similar) values changing as you move the robot over the line.

Tip: if values look reversed (line shows as 0 instead of 1), that’s fine — the line follower below auto-flips.

In [ ]:
for _ in range(20):
    vals = ir.read_sensor_data()
    print(vals)
    time.sleep(0.2)

## 2) Line follow

Tuning knobs:
- `FWD_SPEED`: forward speed (start lower)
- `Kp`: how hard it corrects
- `Kd`: damping (helps on curves / stops oscillation)
- `MAX_YAW`: clamp the turn rate
- `LOST_SPIN_YAW`: how hard to rotate when line is lost

If it wobbles: lower `Kp` or raise `Kd`.
If it misses curves: raise `Kp` or `MAX_YAW`.


In [ ]:
# ---- tuning (start conservative) ----
FWD_SPEED = 70          # 40..120 typical
Kp = 38.0               # proportional gain
Kd = 14.0               # derivative gain
MAX_YAW = 65.0          # clamp yaw correction
LOST_SPIN_YAW = 28.0    # when line lost, rotate to reacquire
DT = 0.02               # loop time target (50Hz)
RUN_SECONDS = 30        # stop after this many seconds

# weights across sensors left->right (works for 3,4,5 sensors too)
def weights_for(n):
    # e.g. n=4 -> [-1.5,-0.5,0.5,1.5]
    mid = (n - 1) / 2.0
    return [i - mid for i in range(n)]

def normalise_sensor_values(vals):
    # Convert to 0/1 ints as best effort
    out = []
    for v in vals:
        if isinstance(v, bool):
            out.append(1 if v else 0)
        else:
            try:
                out.append(1 if int(v) != 0 else 0)
            except Exception:
                out.append(0)
    return out

def infer_line_is_one(sample_patterns):
    # Heuristic:
    # If we see mostly-ones often, likely 'white=1'. If we see sparse ones often, likely 'line=1'.
    # We'll decide based on average sum.
    if not sample_patterns:
        return True
    n = len(sample_patterns[0])
    avg = sum(sum(p) for p in sample_patterns) / (len(sample_patterns) * n)
    # avg near 1.0 means many ones most of the time -> probably background=1, so line=0
    # avg near 0.0 means many zeros most of the time -> probably background=0, so line=1
    # threshold around 0.5
    return avg < 0.5

def compute_error(line_bits, w):
    # Weighted average of active sensors.
    s = sum(line_bits)
    if s == 0:
        return None
    return sum(b*wi for b, wi in zip(line_bits, w)) / s

# ---- quick polarity probe ----
samples = []
print('Sampling sensors for polarity guess... (lift + move it across line if you can)')
for _ in range(25):
    vals = normalise_sensor_values(ir.read_sensor_data())
    if not samples:
        n = len(vals)
        w = weights_for(n)
    samples.append(vals)
    time.sleep(0.05)

LINE_IS_ONE = infer_line_is_one(samples)
print('Detected sensor length:', len(samples[0]))
print('Heuristic: line_is_one =', LINE_IS_ONE, '(if wrong, flip LINE_IS_ONE manually)')

# ---- run controller ----
last_err = 0.0
last_t = time.time()
lost_count = 0
start = time.time()

print('Starting line follow... Ctrl+C to stop')
try:
    while True:
        now = time.time()
        if now - start > RUN_SECONDS:
            print('Time limit reached')
            break

        raw = normalise_sensor_values(ir.read_sensor_data())
        # Convert raw -> line_bits
        if LINE_IS_ONE:
            line_bits = raw
        else:
            line_bits = [0 if b else 1 for b in raw]

        err = compute_error(line_bits, w)
        dt = max(1e-3, now - last_t)
        last_t = now

        if err is None:
            # lost the line: rotate in last known direction until we see it again
            lost_count += 1
            yaw = LOST_SPIN_YAW if last_err > 0 else -LOST_SPIN_YAW
            motors.set_velocity(0, 0, yaw)
        else:
            lost_count = 0
            derr = (err - last_err) / dt
            last_err = err

            yaw = (Kp * err) + (Kd * derr)
            if yaw > MAX_YAW: yaw = MAX_YAW
            if yaw < -MAX_YAW: yaw = -MAX_YAW

            # Forward direction = 0 degrees
            motors.set_velocity(FWD_SPEED, 0, yaw)

        # Print occasionally so we can see behaviour without spamming
        if int((now - start) * 10) % 10 == 0:  # ~1 Hz
            print('raw=', raw, ' line=', line_bits, ' err=', err, ' lost=', lost_count)

        # Loop timing
        sleep_for = DT - (time.time() - now)
        if sleep_for > 0:
            time.sleep(sleep_for)

finally:
    stop()
    print('Stopped')

## Quick tuning cheats

- If it **drifts off on curves**: raise `Kp` (and maybe `MAX_YAW`).
- If it **wobbles**: lower `Kp` or raise `Kd`.
- If it **hunts** and overshoots: raise `Kd` slightly.
- If it **can’t reacquire after losing the line**: increase `LOST_SPIN_YAW`.

If your sensor polarity guess was wrong, set:
- `LINE_IS_ONE = True`  if **1 means line**
- `LINE_IS_ONE = False` if **0 means line**
